데이터 다운로드

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")

print("Path to dataset files:", path)

In [ ]:
ls /kaggle/input/dog-and-cat-classification-dataset

In [ ]:
# 패키지 임포트
import os
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from PIL import Image


In [ ]:
#pytorch 버전 확인
import torch
print(torch.__version__)

In [ ]:
# GPU 사용 체크
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True

파이토치 데이터셋 클래스

In [ ]:
class PyTorchCustomDataset(Dataset):
    def __init__(self,
                 dataset_dir="/kaggle/input/dog-and-cat-classification-dataset/PetImages/",
                 transform=None,
                 max_images=700):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.max_images = max_images

        self.label_list = os.listdir(self.dataset_dir)
        print(f"label_list: {len(self.label_list)}")
        self.label_list.sort()

        self.x_list = []
        self.y_list = []

        total_count = 0

        for label_index, label_str in enumerate(self.label_list):
            img_path = os.path.join(self.dataset_dir, label_str)
            if not os.path.isdir(img_path):
                continue

            img_list = os.listdir(img_path)
            print(f"{label_index} raw image count: {len(img_list)}")

            for img in img_list:
                if total_count >= self.max_images:
                    break

                img_full_path = os.path.join(img_path, img)
                if not os.path.isfile(img_full_path):
                    continue

                # PetImages 데이터셋은 깨진 파일이 섞여있을 수 있어 verify로 걸러줌
                try:
                    with Image.open(img_full_path) as im:
                        im.verify()
                except Exception:
                    continue

                self.x_list.append(img_full_path)
                self.y_list.append(label_index)
                total_count += 1

            if total_count >= self.max_images:
                break

        print(f"[Dataset Info] Loaded images: {len(self.x_list)} (max={self.max_images})")
        print(f"[Dataset Info] Classes: {self.label_list}")

    def __len__(self):
        return len(self.x_list)

    def __getitem__(self, idx):
        image_path = self.x_list[idx]
        label = self.y_list[idx]

        image = Image.open(image_path)
        if image.mode != "RGB":
            image = image.convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __save_label_map__(self, dst_text_path="label_map.txt"):
        with open(dst_text_path, "w") as f:
            for label in self.label_list:
                f.write(label + "\n")

    def __num_classes__(self):
        return len(self.label_list)


In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset_dir = "/kaggle/input/dog-and-cat-classification-dataset/PetImages/"
train_ratio = 0.8
BATCH_SIZE = 32  # 필요 시 사용자가 바꾸면 됨

In [ ]:
dataset = PyTorchCustomDataset(dataset_dir=dataset_dir, transform=transform, max_images=2000)
dataset.__save_label_map__()
num_classes = dataset.__num_classes__()

train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size

print(f"train_size: {train_size}, test_size: {test_size}")


In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4
)

Model

In [ ]:
# 네트워크 정의
import torch
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

class MODEL(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.network = models.resnet18(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Dropout()
            , nn.Linear(1000, num_classes)
            , nn.Sigmoid()
        )
    def forward(self, x):
        x = self.network(x)
        return self.classifier(x)

main 함수

In [ ]:
# 훈련 메인 함수 정의
import torch
import torch.optim as optim

train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

def trainmain():
    USE_CUDA = torch.cuda.is_available()
    DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
    img_width, img_height = 224, 224
    EPOCHS     = 12

    #모델 객체 생성, PyTorch_Classification_Model.pt 모델 파일명 지정
    model = MODEL(num_classes).to(DEVICE)
    model_str = "PyTorch_Classification_Model"
    model_str += ".pt"

    #최적화 함수와 학습률 지정
    #optimizer = optim.Adam(model.parameters(), lr=0.0001)
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    acc = 0.0

    # 에포크 만큼 훈련, 검증
    for epoch in range(1, EPOCHS + 1):
        model.train()
        tr_loss = 0.0
        tr_correct = 0.0
        for data, target in (train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = F.cross_entropy(output, target)
            tr_loss += F.nll_loss(output,target,reduction='sum').item()
            pred = output.data.max(dim=1,keepdim=True)[1]
            tr_correct += pred.eq(target.view_as(pred)).sum().item()
            loss.backward()
            optimizer.step()
        scheduler.step()
        tr_ep_loss = tr_loss/len(train_loader.dataset)
        tr_ep_accuracy = 100. * tr_correct/len(train_loader.dataset)

        model.eval()
        te_loss = 0
        te_correct = 0
        with torch.no_grad():
            for data, target in (test_loader):
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                loss = F.cross_entropy(output, target)
                te_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                te_correct += pred.eq(target.view_as(pred)).sum().item()
        te_ep_loss = te_loss / len(test_loader.dataset)
        te_ep_accuracy = 100. * te_correct / len(test_loader.dataset)
        print('[{}] Train Loss: {:.4f}, Train Accuracy: {:.2f}% Test Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(
                epoch, tr_ep_loss, tr_ep_accuracy, te_ep_loss, te_ep_accuracy))

        if acc < te_ep_accuracy:
            acc = te_ep_accuracy
            torch.save(model.state_dict(), model_str)
            print("model saved!")

        train_losses.append(tr_ep_loss)
        train_accuracy.append(tr_ep_accuracy)
        val_losses.append(te_ep_loss)
        val_accuracy.append(te_ep_accuracy)


In [ ]:
# 훈련 메인 함수 호출
trainmain()

In [ ]:
# 훈련 데이터와 검증 데이터의 손실 그래프
import matplotlib.pyplot as plt

plt.plot(range(1,len(train_losses)+1),train_losses,'bo',label = 'training loss')
plt.plot(range(1,len(val_losses)+1),val_losses,'r',label = 'validation loss')
plt.legend()

In [ ]:
# 훈련 데이터와 검증 데이터의 정확도 그래프
plt.plot(range(1,len(train_accuracy)+1),train_accuracy,'bo',label = 'train accuracy')
plt.plot(range(1,len(val_accuracy)+1),val_accuracy,'r',label = 'val accuracy')
plt.legend()

In [ ]:
# PyTorch_Classification_Model.pt 모델 파일 확인
!ls

이미지 분류 모델 추론 테스트

In [ ]:
# 테스트 이미지 로딩
import os
#PATH = "/content/cats_and_dogs_filtered/validation"
#validation_cats_dir = PATH + '/cats'  # directory with our validation cat pictures
#validation_dogs_dir = PATH + '/dogs'  # directory with our validation dog pictures
PATH = "/kaggle/input/dog-and-cat-classification-dataset/PetImages/"
validation_cats_dir = PATH + '/Cat'  # directory with our validation cat pictures
validation_dogs_dir = PATH + '/Dog'  # directory with our validation dog pictures

list_of_test_cats_images = os.listdir(validation_cats_dir)
list_of_test_dogs_images = os.listdir(validation_dogs_dir)
for idx in range(len(list_of_test_cats_images)):
    list_of_test_cats_images[idx] = validation_cats_dir + '/'+list_of_test_cats_images[idx]
for idx in range(len(list_of_test_dogs_images)):
    list_of_test_dogs_images[idx] = validation_dogs_dir + '/'+list_of_test_dogs_images[idx]
list_of_test_images = list_of_test_cats_images + list_of_test_dogs_images

In [ ]:
# 로딩된 이미지 경로 프린트
print(list_of_test_cats_images[10])
print(list_of_test_images[501])

In [ ]:
# 이미지 보여주는 함수, 이미지 추론 함수

from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

#라벨맵 로딩 함수
def load_label_map(textFile):
    return np.loadtxt(textFile, str, delimiter='\t')

#이미지 읽는 함수
def cv_image_read(image_path):
    print(image_path)
    return cv2.imread(image_path)

#이미지 보여주는 함수
def show_image(cv_image):
    rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(rgb)
    plt.show()

#이미지를 분류 모델로 추론한 결과를 텍스트로 보여주는 함수
def print_result(inference_result, class_map):
    class_text = class_map[np.argmax(inference_result)]
    print(inference_result)
    print(class_text)

#이미지를 분류 모델로 추론하는 함수
def inference_image(opencv_image, transform_info, model, DEVICE):
    image = Image.fromarray(opencv_image)
    image_tensor = transform_info(image)
    image_tensor = image_tensor.unsqueeze(0)
    image_tensor = image_tensor.to(DEVICE)
    result = model(image_tensor)
    return result

In [ ]:
# 이미지 보여주는 함수 실습
show_image(cv_image_read(list_of_test_images[10]))
show_image(cv_image_read(list_of_test_images[501]))

In [ ]:
# 테스트 메인 함수
import os
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

def testmain(image_path):
    USE_CUDA = torch.cuda.is_available()
    DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

    img_width, img_height = 224, 224
    transform_info = transforms.Compose([
                transforms.Resize(size=(img_width, img_height))
                , transforms.ToTensor()
                    ])
    #라벨 파일 읽기
    class_map = load_label_map('label_map.txt')
    num_classes = len(class_map)

    #지정된 모델 로딩
    model = MODEL(num_classes).to(DEVICE)
    model_str = "PyTorch_Classification_Model"
    model_str += ".pt"

    model.load_state_dict(torch.load(model_str))
    model.eval()

    #image_path = list_of_test_images[501]
    opencv_image = cv_image_read(image_path)
    inference_result = inference_image(opencv_image, transform_info, model, DEVICE)
    inference_result = inference_result.cpu().detach().numpy()
    print_result(inference_result, class_map)
    show_image(opencv_image)

In [ ]:
# 테스트 이미지로 테스트 메인 함수 실행 1
image_path = list_of_test_images[10]
testmain(image_path)

# 테스트 이미지로 테스트 메인 함수 실행 2
image_path = list_of_test_images[501]
testmain(image_path)